# 安装核心依赖

```bash
# pip3 install sentence-transformers chromadb transformers torch numpy openai
```

# ====================== 模块1：Embedding模型封装（统一接口，支持灵活扩展） ======================

In [3]:
from abc import ABC, abstractmethod
import numpy as np
from typing import List, Union
from sentence_transformers import SentenceTransformer
import openai  # 可选：云端Embedding模型依赖

# 1. 定义Embedding抽象基类（统一接口，新增模型只需实现该接口）
class EmbeddingBase(ABC):
    @abstractmethod
    def embed_text(self, texts: Union[str, List[str]], normalize: bool = True) -> np.ndarray:
        """
        文本向量化核心方法
        :param texts: 单个文本字符串或文本列表
        :param normalize: 是否归一化向量（提升检索相似度计算精度）
        :return: 向量数组（shape: [文本数量, 向量维度]）
        """
        pass

    @property
    @abstractmethod
    def vector_dim(self) -> int:
        """返回模型生成的向量维度（用于Chroma向量库初始化）"""
        pass

# 2. 本地开源Embedding模型实现（以BAAI/bge-base-zh-v1.5为例，中文比较优选择）
class LocalBgeEmbedding(EmbeddingBase):
    def __init__(self, model_name: str = "BAAI/bge-base-zh-v1.5", device: str = None):
        """
        初始化本地Embedding模型
        :param model_name: 模型名称（Hugging Face仓库）或本地模型路径
        :param device: 运行设备（None自动选择，"cuda"=GPU，"cpu"=CPU）
        """
        # 加载模型（支持本地路径/远程仓库）
        self.model = SentenceTransformer(model_name, device=device)
        self._vector_dim = None  # 延迟初始化向量维度

    def embed_text(self, texts: Union[str, List[str]], normalize: bool = True) -> np.ndarray:
        # 处理单个文本输入
        if isinstance(texts, str):
            texts = [texts]
        
        # 生成向量：normalize=True是关键优化（提升检索效果）
        embeddings = self.model.encode(
            texts,
            convert_to_numpy=True,  # 转为numpy数组
            normalize_embeddings=normalize,  # 向量归一化
            show_progress_bar=False  # 关闭进度条
        )
        
        # 自动获取向量维度（仅首次调用时初始化）
        if self._vector_dim is None:
            self._vector_dim = embeddings.shape[1]
        return embeddings

    @property
    def vector_dim(self) -> int:
        # 若未初始化，通过测试文本获取向量维度
        if self._vector_dim is None:
            self.embed_text("测试文本")
        return self._vector_dim

# 3. 可选：云端Embedding模型实现（以OpenAI为例，无需本地算力）
class OpenAIEmbedding(EmbeddingBase):
    def __init__(self, api_key: str, model_name: str = "text-embedding-3-small"):
        """
        初始化OpenAI Embedding API
        :param api_key: OpenAI API密钥
        :param model_name: Embedding模型名称
        """
        self.client = openai.OpenAI(api_key=api_key)
        self.model_name = model_name
        self._vector_dim = None

    def embed_text(self, texts: Union[str, List[str]], normalize: bool = True) -> np.ndarray:
        if isinstance(texts, str):
            texts = [texts]
        
        # 调用OpenAI API生成向量
        response = self.client.embeddings.create(
            input=texts,
            model=self.model_name
        )
        
        # 提取向量并转为numpy数组
        embeddings = [item.embedding for item in response.data]
        embeddings_np = np.array(embeddings)
        
        # 向量归一化（可选，提升检索精度）
        if normalize:
            embeddings_np = embeddings_np / np.linalg.norm(embeddings_np, axis=1, keepdims=True)
        
        # 初始化向量维度
        if self._vector_dim is None:
            self._vector_dim = embeddings_np.shape[1]
        return embeddings_np

    @property
    def vector_dim(self) -> int:
        if self._vector_dim is None:
            self.embed_text("测试文本")
        return self._vector_dim

# 初始化本地Embedding模型（优先使用，无需API Key，隐私安全）
# 若网络无法访问Hugging Face，可改为本地模型路径（如"./local_models/bge-base-zh-v1.5"）
local_embedding = LocalBgeEmbedding(device=None)
print(f"✅ 本地Embedding模型初始化完成，向量维度：{local_embedding.vector_dim}")

d:\soft\py\py_3_11_9\Lib\site-packages\torch\_utils.py:776: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


✅ 本地Embedding模型初始化完成，向量维度：768


# ====================== 模块2：Chroma向量数据库封装（后端存储实现，适配新版Chroma） ======================

In [4]:
import chromadb
from chromadb.config import Settings
import os
from typing import Optional, Dict

class ChromaVectorStore:
    def __init__(
        self,
        embedding_model: EmbeddingBase,
        collection_name: str = "rag_knowledge_base",
        persist_directory: str = "./chroma_persist",
        metadata: Optional[Dict] = None
    ):
        """
        初始化Chroma向量数据库（持久化模式，新版自动持久化，无需手动persist）
        :param embedding_model: 已实例化的Embedding模型（用于向量生成）
        :param collection_name: 集合名称（对应一个独立知识库）
        :param persist_directory: 持久化存储目录（数据保存在本地文件夹）
        :param metadata: 集合元数据（如知识库描述）
        """
        self.embedding_model = embedding_model
        self.collection_name = collection_name
        self.persist_directory = persist_directory

        # 1. 初始化Chroma持久化客户端（新版自动持久化，无需后续手动调用persist）
        self.client = chromadb.PersistentClient(
            path=persist_directory,
            settings=Settings(
                anonymized_telemetry=False,  # 关闭匿名数据上报
                allow_reset=True  # 允许重置集合（仅测试用）
            )
        )

        # 2. 获取或创建集合（相当于数据库中的"表"）
        self.collection = self.client.get_or_create_collection(
            name=collection_name,
            metadata=metadata or {"description": "RAG框架知识库集合"}
        )

    def add_documents(
        self,
        documents: List[str],
        ids: Optional[List[str]] = None,
        metadatas: Optional[List[Dict]] = None
    ):
        """
        向Chroma添加知识库文档（自动生成向量并自动持久化）
        :param documents: 文本文档列表（知识库内容）
        :param ids: 文档唯一ID列表（可选，不指定则自动生成）
        :param metadatas: 文档元数据列表（可选，如{"source": "PDF文件1"}）
        """
        # 1. 用Embedding模型生成文档向量（转为列表格式，适配Chroma）
        embeddings = self.embedding_model.embed_text(documents).tolist()

        # 2. 自动生成文档ID（若未指定）
        if ids is None:
            current_count = self.collection.count()
            ids = [f"doc_{current_count + i + 1}" for i in range(len(documents))]

        # 3. 添加文档、向量、元数据到集合（新版Chroma自动持久化到本地目录）
        self.collection.add(
            documents=documents,
            embeddings=embeddings,
            ids=ids,
            metadatas=metadatas
        )

        # 新版无需手动调用persist()，删除过时代码
        print(f"✅ 成功添加{len(documents)}条文档，当前知识库总文档数：{self.collection.count()}")

    def similarity_search(
        self,
        query: str,
        top_k: int = 3,
        where: Optional[Dict] = None
    ) -> List[Dict]:
        """
        相似文本检索（核心：根据用户查询找到最相关的知识库内容）
        :param query: 用户查询语句
        :param top_k: 返回最相似的k条文档
        :param where: 元数据过滤条件（如{"source": "电商订单"}）
        :return: 格式化检索结果（含文档、相似度、ID、元数据）
        """
        # 1. 生成查询向量
        query_embedding = self.embedding_model.embed_text(query).tolist()

        # 2. Chroma相似性检索（返回距离和文档信息）
        results = self.collection.query(
            query_embeddings=query_embedding,
            n_results=top_k,
            where=where,
            include=["documents", "distances", "metadatas"]  # 指定返回字段
        )

        # 3. 格式化结果（距离转相似度，0-1之间，数值越大越相似）
        formatted_results = []
        for i in range(top_k):
            if i >= len(results["ids"][0]):  # 防止结果不足top_k
                break
            formatted_results.append({
                "id": results["ids"][0][i],
                "document": results["documents"][0][i],
                "similarity": round(1 / (1 + results["distances"][0][i]), 4),  # 距离转相似度
                "metadata": results["metadatas"][0][i] if results["metadatas"] else None
            })
        return formatted_results

    def get_collection_stats(self) -> Dict:
        """获取知识库统计信息"""
        return {
            "collection_name": self.collection_name,
            "document_count": self.collection.count(),
            "vector_dim": self.embedding_model.vector_dim,
            "persist_directory": self.persist_directory
        }

# 初始化Chroma向量存储（注入本地Embedding模型，新版自动持久化）
chroma_store = ChromaVectorStore(
    embedding_model=local_embedding,
    collection_name="my_custom_rag_kb",
    persist_directory="./chroma_persist"
)
print(f"✅ Chroma向量数据库初始化完成，统计信息：{chroma_store.get_collection_stats()}")

✅ Chroma向量数据库初始化完成，统计信息：{'collection_name': 'my_custom_rag_kb', 'document_count': 20, 'vector_dim': 768, 'persist_directory': './chroma_persist'}


# ====================== 模块3：完整RAG框架整合（检索+生成） ======================

In [5]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

class CustomRAGSystem:
    def __init__(
        self,
        embedding_model: EmbeddingBase,
        chroma_store: ChromaVectorStore,
        llm_model_name: str = "Qwen/Qwen2-0.5B-Instruct"  # 轻量LLM，CPU可运行
    ):
        """
        初始化完整RAG系统
        :param embedding_model: Embedding模型实例
        :param chroma_store: Chroma向量存储实例
        :param llm_model_name: LLM模型名称（Hugging Face）或本地路径
        """
        self.embedding_model = embedding_model
        self.chroma_store = chroma_store

        # 自动选择运行设备（有GPU用GPU，无GPU用CPU）
        self.device = "cuda" if torch.cuda.is_available() else "cpu"
        print(f"📥 正在加载LLM模型：{llm_model_name}，运行设备：{self.device}")

        # 1. 初始化LLM分词器（将文本转为模型可识别的编码）
        self.tokenizer = AutoTokenizer.from_pretrained(
            llm_model_name,
            trust_remote_code=True,  # 信任自定义模型代码
            padding_side="right"  # 右侧填充（优化生成效果）
        )

        # 2. 初始化LLM模型（生成回答的核心）
        self.llm = AutoModelForCausalLM.from_pretrained(
            llm_model_name,
            device_map=self.device,  # 自动分配设备
            torch_dtype=torch.float16 if self.device == "cuda" else torch.float32,  # 适配精度
            trust_remote_code=True
        )
        self.llm.eval()  # 切换到推理模式（关闭训练相关功能）
        print(f"✅ LLM模型加载完成")

    def load_knowledge_base(self, knowledge_texts: List[str], metadatas: Optional[List[Dict]] = None):
        """加载知识库到Chroma向量库"""
        self.chroma_store.add_documents(documents=knowledge_texts, metadatas=metadatas)

    def _build_prompt(self, query: str, retrieved_docs: List[Dict]) -> str:
        """
        构建RAG提示词（核心：引导LLM基于知识库回答，避免幻觉）
        :param query: 用户问题
        :param retrieved_docs: 检索到的相似文档
        :return: 格式化提示词
        """
        # 拼接知识库内容
        context = "\n\n".join([f"【知识库{idx+1}】：{doc['document']}" for idx, doc in enumerate(retrieved_docs)])
        
        # 提示词模板（清晰约束LLM行为，提升回答质量）
        prompt_template = """
请严格基于以下知识库内容回答用户问题，不要引入任何知识库外的信息。
如果知识库中没有相关信息，请明确回复“未检索到相关知识，无法回答”。

### 知识库内容：
{context}

### 用户问题：
{query}

### 回答：
        """
        return prompt_template.format(context=context, query=query).strip()

    def generate_answer(self, query: str, top_k: int = 3, temperature: float = 0.7) -> Dict:
        """
        生成RAG回答（端到端流程：检索→构建提示词→LLM生成）
        :param query: 用户问题
        :param top_k: 检索相似文档数量
        :param temperature: 生成随机性（0=确定性回答，1=最大随机性）
        :return: 格式化回答结果
        """
        # 步骤1：相似文档检索
        retrieved_docs = self.chroma_store.similarity_search(query, top_k=top_k)
        print(f"🔍 检索到{len(retrieved_docs)}条相似文档")

        # 步骤2：构建提示词
        prompt = self._build_prompt(query, retrieved_docs)

        # 步骤3：LLM生成回答
        inputs = self.tokenizer(prompt, return_tensors="pt").to(self.device)  # 转为张量并移至指定设备
        with torch.no_grad():  # 关闭梯度计算，节省显存/内存
            outputs = self.llm.generate(
                **inputs,
                temperature=temperature,
                max_new_tokens=512,  # 最大生成字数
                pad_token_id=self.tokenizer.eos_token_id,  # 填充token
                eos_token_id=self.tokenizer.eos_token_id  # 结束token
            )

        # 步骤4：解析回答（去除提示词部分，只保留回答内容）
        full_text = self.tokenizer.decode(outputs[0], skip_special_tokens=True)
        answer = full_text.split("### 回答：")[-1].strip()

        # 返回格式化结果
        return {
            "user_query": query,
            "retrieved_documents": retrieved_docs,
            "rag_answer": answer,
            "top_k": top_k
        }

# 实例化完整RAG系统
rag_system = CustomRAGSystem(
    embedding_model=local_embedding,
    chroma_store=chroma_store,
    llm_model_name="Qwen/Qwen2-0.5B-Instruct"
)

📥 正在加载LLM模型：Qwen/Qwen2-0.5B-Instruct，运行设备：cpu
✅ LLM模型加载完成


# ====================== 模块4：测试运行（验证RAG功能） ======================

In [6]:
# 1. 准备测试知识库（可替换为你的业务知识）
test_knowledge = [
    "BAAI/bge-base-zh是北京人工智能研究院发布的中文开源Embedding模型，向量维度768，适合中文RAG检索场景。",
    "Chroma是轻量级本地向量数据库，支持持久化存储和元数据过滤，无需复杂部署，适合私有化RAG框架。",
    "RAG框架核心三步：知识库向量化存储、用户查询相似检索、结合检索结果生成回答。",
    "Embedding模型的作用是将文本转为低维稠密向量，通过余弦相似度判断文本语义相似性。",
    "Qwen2是阿里云发布的轻量级大语言模型，支持中文流畅生成，适合本地低算力设备运行。"
]

# 2. 加载知识库到Chroma
rag_system.load_knowledge_base(
    knowledge_texts=test_knowledge,
    metadatas=[{"source": "测试知识库"} for _ in test_knowledge]
)

# 3. 测试用户查询
user_queries = [
    "BAAI/bge-base-zh适合什么场景？",
    "Chroma向量数据库的特点是什么？",
    "RAG框架的核心步骤有哪些？",
    "Embedding模型的作用是什么？"
]

# 4. 批量生成回答并输出
print("\n" + "="*60)
print("🎯 RAG问答测试开始")
print("="*60)
for query in user_queries:
    result = rag_system.generate_answer(query, top_k=2)
    print(f"\n📝 用户问题：{result['user_query']}")
    print(f"🤖 RAG回答：{result['rag_answer']}")
    print(f"📊 检索文档数量：{len(result['retrieved_documents'])}")
    print("-"*50)

# 5. 输出知识库最终统计
print(f"\n✅ 测试完成，知识库统计：{chroma_store.get_collection_stats()}")

✅ 成功添加5条文档，当前知识库总文档数：25

🎯 RAG问答测试开始
🔍 检索到2条相似文档

📝 用户问题：BAAI/bge-base-zh适合什么场景？
🤖 RAG回答：
📊 检索文档数量：2
--------------------------------------------------
🔍 检索到2条相似文档

📝 用户问题：Chroma向量数据库的特点是什么？
🤖 RAG回答：
📊 检索文档数量：2
--------------------------------------------------
🔍 检索到2条相似文档

📝 用户问题：RAG框架的核心步骤有哪些？
🤖 RAG回答：RAG框架的核心步骤有：

知识库1：RAG框架的核心三步包括：知识库向量化存储、用户查询相似检索和结合检索结果生成回答。这些步骤是RAG框架的三个关键环节，它们共同构成了RAG框架的核心功能。

知识库2：RAG框架的核心三步还包括：用户查询相似检索、结合检索结果生成回答以及知识库向量化存储。这三个步骤构成了RAG框架的基本框架，其中，用户查询相似检索负责提供与用户搜索相关的信息；结合检索结果生成回答则根据用户的查询结果，为用户提供一个更准确的答案；而知识库向量化存储则用于将大量的知识数据进行规范化处理，以便更好地服务于用户。因此，RAG框架的核心步骤是通过这三个环节，实现对大量信息的高效管理和快速响应。
📊 检索文档数量：2
--------------------------------------------------
🔍 检索到2条相似文档

📝 用户问题：Embedding模型的作用是什么？
🤖 RAG回答：Embedding模型是一种用于表示文本的稀疏表示方法，它使用邻接矩阵来描述文本和它的上下文。在这种情况下，我们可以将文本看作是一个高维空间，而其上下文则被定义为一个二维数组，其中每一行代表一个元素（单词或句子），每列代表一个特征值。这种做法使得我们能够利用余弦相似度计算两个文本之间的语义相似度。

因此，Embedding模型的主要作用是在文本处理和机器翻译等领域扮演着重要的角色，它可以将文本转换成更易于理解和表达的形式，并且可以用来衡量文本之间的语义相似度。
📊 检索文档数量：2
---------------------------------